# Batteries Included: An opinionated toolset to get started analyzing your data
## Do it live: Jupyter Notebook and IPython
"The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text."

* Inline documentation - jump into definitions, parameter lists, etc.
* Instant feedback - see results of executed code blocks immediately
* Quick iteration - change code and rerun it without changing parts that already work
* Lots of language support (we're going to focus on Python)
* Tab completion - great if you can't type, like Mat

Learn more at the [Jupyter website](https://jupyter.org/)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipaddress
import random
import geopandas as gpd
%matplotlib inline

## Manage your data: Pandas (and Numpy)
"pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."

* Load data from many sources - relational databases, text (csv, json, ...), excel, hdfs, ...
  * Can be loaded locally or from a URL
* Indexing - quick queries, rollups, ...
* SQL-like operations - grouping, ordering, operators, ...
* Plays well with others - lots of integrations for things like visualization, stats, machine learning, ...

Learn more at [the pandas website](https://pandas.pydata.org/)

In [ ]:
# The result of a loading operation is a DataFrame, one of the core pandas data structures
df = pd.read_json("https://m3aawg47.s3.amazonaws.com/workbook/events.jsonl.gz", lines=True)

In [ ]:
df.describe()

---

# Easiest: Stateless rules
Stateless rules use only information within the single event to make a decision.

>    Example: Flag accounts who send emails with > X recipients.

An offline process (like this one!) can help discover limits for stateless rules. 

The next cells will work the example given above: identifying accounts sending to a large number of recipients.

In [ ]:
# Pandas has built-in simple statistics
df["nr_recipients"].describe()

In [ ]:
# Pandas also has built-in graphing via matplotlib
df["nr_recipients"].hist()
plt.show()

In [ ]:
# Filtering operations return a new DataFrame (actually, a view into the original one)
df[df.nr_recipients >= 60].head()

In [ ]:
# Selecting a single column returns a Series, the other fundamental pandas data structure
df[df.nr_recipients >= 60]['user_id'].unique()

## CTF Challenge 1: Find all the users who sent emails that both a) had more than 10 recipients AND b) had an attached EXE file
Hint: Check out [this section](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) of the pandas docs.

---

# Slightly less easy: Joined data
Beyond the simplest, stateless rules, you can get a lot of power by joining external data sources in.

> Example: Geolocating source IP addresses for logins

With data joined in, you can then apply otherwise stateless rules that leverage the joined data.

> Example: Flag logins originating from an IP in the threat intelligence list

The next cells will look at geolocating data and using visualization to identify interesting accounts.

In [ ]:
# Don't worry, this won't be on the test

# Load the geo data, then set up an "interval index" - a way to quickly check
# if a number is between the two numbers in the interval (in this case, low and high IP
# addresses)
geo_df = pd.read_csv('https://m3aawg47.s3.amazonaws.com/workbook/simple-geo.csv.gz')
v = list(zip(geo_df.lo_addr, geo_df.hi_addr))
idx = pd.IntervalIndex.from_tuples(v, closed='both')
geo_df.set_index(idx, inplace=True)
geo_df.head()

In [ ]:
import functools

@functools.lru_cache()
def lookup_ip_geo(ip):
    addr = int(ipaddress.ip_address(ip))
    return geo_df.loc[addr][['latitude', 'longitude', 'country_iso_code', 'city_name']]

# Enhance our original DataFrame by joining the geolocation data!
# This is going to take about 30s
df[['latitude', 'longitude', 'country_iso_code', 'city_name']] = df.src_ip.apply(lookup_ip_geo)
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(figsize=(16, 16))
gdf.plot(ax=base, marker='o', color='red', markersize=10);

In [ ]:
# WHOA! THERE ARE PEOPLE OUTSIDE THE US!?
df[df['country_iso_code'] != 'US']['user_id'].unique()

## CTF Challenge 2: Find all accounts that had an event with either a source or destination IP in the threat list http://example.com/threat-list.fillmein.csv
Hint: check out these links:
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html#pandas.Series.apply
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge
---

# Intermediate: Partitioned state
Collecting and comparing historical state opens the door to finding *changes* in behavior rather than specific behaviors. This enables a set of powerful rules that require less human effort in creation and maintenance, since they operate on entity-specific baselines rather than static rules.

> Example: Collecting send rates by hour for each account

We say that this state is *partitioned* because the state is scoped to a specific entity.

With the state available for query, you can build analytics against it to illuminate interesting (or risky) changes in behavior.

> Example: Flag accounts with an hourly sending volume three times their average

The next cells will identify users that had a significant change in their daily sending volume.

In [ ]:
df['user_id'].value_counts()

In [ ]:
big_user = df.loc[df.user_id == '010683f9-d720-4ba1-ae6d-341c1a92b3f2']
days = big_user.timestamp.dt.round('d')
big_counts = big_user.groupby(days).user_id.count()
big_counts.plot()
big_counts.expanding().mean().plot()
plt.show()

In [ ]:
# pandas Series includes vector-valued comparisons (and other operators)
big_counts > 3 * big_counts.expanding(7).mean()

In [ ]:
for user_id in list(df.user_id.unique()):
    big_user = df.loc[df.user_id == user_id]
    hours = big_user.timestamp.dt.round('d')
    big_counts = big_user.groupby(hours).user_id.count()
    if any(big_counts > 3 * big_counts.expanding(7).mean()):
        print(user_id)

## CTF Challenge 3: Find the three users with the largest change in attachment behavior. In other words, find the three users who had the biggest increase in number of attachments per day. Is their behavior significantly different from the rest of the population?

Hint: You can add a new column to a dataframe like this:

    df['new_column_name'] = new_series
    
This is commonly used when you want to compute a new column from an existing column. For example, given our DataFrame from this workbook, you could do something like this:

    df['twice_recipients'] = df['recipients'].map(lambda x: x * 2)


---

# Difficult: Unpartitioned state
Recall from the last section that we say state is *partitioned* to mean that we are only tracking state on a per-entity basis. Analytics based on unpartitioned state, in contrast, need to look across entities to perform their function.

> Example: Finding send rates outside of 3 standard deviations from the average across all users

Operating on a full dataset, as we have in this notebook, actually *is* leveraging unpartitioned state - we have looked at, for example, the distribution of recipient counts to identify a reasonable threshold for simple rules. Unpartitioned state becomes challenging when it is implemented at scale. Tracking cross-entity state (or generating it from an indexed data store) is difficult to do efficiently. 

A typical implementation looks like what is called a "lambda architecture": a streaming path for fast application of analytics, and a batch-processing path for generation of thresholds or models to use in the streaming path. Building and tuning systems like this, particularly at scale, is a serious undertaking.

---

# CTF Challenge 4: Find the user who was NOT outside of the United States that had the highest rate of travel (i.e., distance between consecutive events / time between consecutive events).

Hint: https://en.wikipedia.org/wiki/Geographical_distance

Please use the "Spherical Earth projected to a plane" formula.

---

# YOU'RE DONE!

You are now officially a data scientist. Well... you're on your way. Good luck, and come talk to us if you have any questions!

---

# _CTF Bonus round_

These problems are a good bit harder.

B1. Each user of the population in this dataset has a set of parameters that defined how the events were generated. For example, each user has an average number of recipients. These per-user parameters are drawn from a Poisson distribution. What is the parameter?

B2. Which users had a significant shift in their daily operational hours?